# Multiline Diagrams

## Object construction

In [1]:
import itertools 
import copy
from sage.combinat.q_analogues import *
from sage.combinat.sf.sf import *
from sage.combinat.ncsf_qsym.qsym import *
from sage.rings.rational_field import *
from sage.combinat.subset import *
from sage.combinat.permutation import *
from sage.misc.latex import *

import sage.combinat.permutation as permutation
from sage.combinat.sf.macdonald import qt_kostka
from sage.combinat.sf.ns_macdonald import E
from sage.rings.polynomial.polydict import ETuple

coeffs_ring = ZZ['q','t']

q = coeffs_ring.gens()[0]
t = coeffs_ring.gens()[1]

coeffs_field = coeffs_ring.fraction_field()
R=PolynomialRing(coeffs_field,11,'x')
xs=list(R.gens())

sym = SymmetricFunctions(coeffs_field)
h = sym.homogeneous()
m = sym.monomial()
s = sym.schur()
e = sym.elementary()
p = sym.power()
Ht = sym.macdonald().Ht()
H = sym.macdonald().H()
W = sym.macdonald(t=0).P()
S = sym.macdonald(q=0).S()

MJ = sym.macdonald().J()
HLP = sym.hall_littlewood(t).P()
HLQ = sym.hall_littlewood(t).Q();
HLQp = sym.hall_littlewood(t).Qp();
JJ = sym.jack().J()
qsym = QuasiSymmetricFunctions(coeffs_field)
F = qsym.Fundamental()
M = qsym.Monomial()
QS = qsym.QS()
YQS = qsym.YQS()

In [2]:
# COORDINATES START AT 1
# Coordinates are given in the usual way (i,j) means i to the right, j up

def balls_coordinates(balls):
    coords = []
    w = len(balls)
    l = len(balls[0])
    for i in range(w):
        for j in reversed(range(l)):
            if balls[i][j] != 0: 
                for k in range(balls[i][j]):
                    coords.append([i+1,j+1])
    return(coords)

def balls_coordinates_right_order(balls):
    coords = []
    w = len(balls)
    l = len(balls[0])
    for i in range(w):
        for j in (range(l)):
            if balls[i][j] != 0: 
                for k in range(balls[i][j]):
                    coords.append([i+1,j+1])
    return(coords)
    
def balls_is_valid(balls):
    return(True)

# find the queue index in which elem is
def find_queue(self,elem):
    ind = -1
    for i in range(len(self.queues)):
        queue = self.queues[i]
        if elem in queue:
            ind = i
            break
    if ind == -1:
        return("element not queued")
    else:
        return(ind)  
    
# word is a ternary word with 0,1,2 and 1=(, 2=) and 0=blank, to perform parenthesis matching algorithm
# returns the leftover letters from row1 after matching all possible 1s and 2s, and deleting unmatched 2s
def match_parenthesis(w):
    word = w.copy()
    boo = True
    while boo:
        p1 = -1
        p2 = -1
        for i in range(len(word)):
            if word[i] == 1:
                p1 = i
            elif word[i] == 2:
                p2 = i
            if p1 != -1:
                if p2 != -1 and p2>p1:
                    word[p1] = 0
                    word[p2] = 0        
                    break
                else:
                    l = [j for j in range(p1,len(word)) if word[j] == 2]
                    if l==[]:
                        word[p1] = 0
                    
        if p1==-1:
            boo = False
    return(word)
    
# collapse row 1 on top of row 2
# this method updates row1 and row2
# wit is a boolean variable that says if a collapse occurred or not
def two_row_collapse(row1,row2):
    par_word = []
    wit = False
    # Create the parenthesis word to do the matching algorithm
    for i in range(len(row1)):
        if row1[i]!=0:
            for k in range(row1[i]):
                par_word.append(2)
        else:
            par_word.append(0)
        if row2[i]!=0:
            for k in range(row2[i]):
                par_word.append(1)
        else:
            par_word.append(0)
    # Find indices of each row balls
    row1_indices = []
    row2_indices = []
    
    for j in range(len(par_word)):
        if par_word[j] == 1:
            row2_indices.append(j)
        elif par_word[j] == 2:
            row1_indices.append(j)
    # Apply parenthesis matching
    red_par_word = match_parenthesis(par_word)
    # Find indices on row1 to collapse
    
    coll_indices = []
    
    for j in range(len(red_par_word)):
        if red_par_word[j] == 2:
            if j in row1_indices:
                ind = row1_indices.index(j)
                coll_ind = min([k for k in range(len(row1)) if sum(row1[:k+1])>ind])
                coll_indices.append(coll_ind)
    for i in coll_indices:
        wit = True
        row1[i] -= 1
        row2[i] += 1
    return(row1.copy(),row2.copy(),wit)

# Print a list of lists in a nice way
def print_list_of_lists(L):
    #print("--------------------------")
    print('\n'.join(' '.join('%2d' % x for x in l) for l in L))
    #print("--------------------------")

##   MULTILINE DIAGRAMS CLASS   ##

class MultilineDiagram:
    queues = []
    case = 0
    r = 0
    set_case = False
     
    
    # balls may be valid (partition content)
    # size = [width,height]
    def __init__(self,balls):
        self.balls = balls
        self.size = [len(balls[0]),len(balls)]
        self.balls_coordinates = balls_coordinates(balls)
        self.QM =[ [] for i in range(sum([sum(balls[i]) for i in range(len(balls))])) ]
        
    # This method creates the queues
    # 
    # case=0:   pairing to the left with (possibly) wrapping conditions WITH NO VERTICAL PAIRING ALLOWED
    #           pairing is done right to left in each row and priority order is respected
    #           queues are given as sequences of coordinates starting from the top
    #
    # case=1:   minimal distance insertion-pairing to the left with no-wrapping and NO VERTICAL PAIRING ALLOWED
    #           collapses ball arrangement and pairs using case 0
    #
    # case=2:   minimal distance insertion-pairing to the left with no-wrapping and NO VERTICAL PAIRING ALLOWED
    #           inserts each ball from right to left and updates the recording tableaux
    #
    # case=3:   pairing to the right with (possibly) wrapping conditions WITH NO VERTICAL PAIRING ALLOWED
    #           pairing is done left to right in each row and priority order is respected
    #           queues are given as sequences of coordinates starting from the top
    #
    # case=4:   minimal distance insertion-pairing to the right with no-wrapping and NO VERTICAL PAIRING ALLOWED
    #           inserts each ball from left to right and updates the recording tableaux
    #
    
    def pair(self,case):
        if not self.set_case:
            self.case = case
            self.set_case = True
        self.queues = []
        
        if case==0:       
            to_queue = self.balls_coordinates.copy()
            cont = True
            while cont:
                b = to_queue[-1]
                queue = [b]
                to_queue.remove(b)
                done = False
                b0 = b.copy()
                while not done:
                    j = b0[0]-1
                    l = [m for m in range(b0[1]) if ([j,m] in to_queue)]
                    #if b0 is at the bottom
                    if b0[0] == 1:
                        done = True
                    #otherwise
                    else:
                        #case 1: wrapping
                        if l==[]:
                            #c = max([m for m in range(b0[1]+1,self.size[0]+1) if ([j,m] in to_queue)])
                            c = max([m for m in range(b0[1],self.size[0]+1) if ([j,m] in to_queue)])
                        #case 2: non-wrapping
                        else:
                            c = max(l)
                        queue.append([j,c])
                        to_queue.remove([j,c])
                        b0 = [j,c]
                self.queues.append(queue)
                if len(to_queue) == 0:
                    cont = False  
        
        if case == 1:
            self.recording_and_collapse()
            while (self.size[0])*[0] in self.balls:
                self.balls.remove((self.size[0])*[0])
            self.size = [len(self.balls[0]),len(self.balls)]
            self.balls_coordinates = balls_coordinates(self.balls)
            self.pair(0)
            
        if case == 2:
            self.recording_and_collapse()
            self.size = [len(self.balls[0]),len(self.balls)]
            self.balls_coordinates = balls_coordinates(self.balls)
            self.pair(0)
            
        if case == 3:
            rev_balls = []
            for b in self.balls:
                rev_balls.append(list(reversed(b)))
            self.balls_coordinates = balls_coordinates(rev_balls)
            self.pair(0)
            
            self.balls_coordinates = balls_coordinates(self.balls)
            rev_queues = []
            for queue in self.queues:
                rev_q = []
                for p in queue:
                    rev_q.append([p[0],self.size[0]+1-p[1]])
                rev_queues.append(rev_q)
            
            self.queues = rev_queues
        
        if case == 4:
            rev_balls = []
            for b in self.balls:
                rev_balls.append(list(reversed(b)))
            self.balls_coordinates = balls_coordinates(rev_balls)
            self.pair(2)
            
            rb = []
            for b in self.balls:
                rb.append(list(reversed(b)))
            self.balls = rb
            self.balls_coordinates = balls_coordinates(self.balls)
            rev_queues = []
            for queue in self.queues:
                rev_q = []
                for p in queue:
                    rev_q.append([p[0],self.size[0]+1-p[1]])
                rev_queues.append(rev_q)
            
            self.queues = rev_queues
            
                
    # Collapsing procedure for the ball arrangement.
    # WARNING: This changes the original balls so that the new arrangement is non-wrapping
    #          
    
    def collapse(self):
        L = self.size[1]
        for i in range(1,L):
            for j in reversed(range(1,i+1)):
                row1 = balls[j-1].copy()
                row2 = balls[j].copy()
                [balls[j],balls[j-1],wit] = two_row_collapse(row2,row1)
    
                
    # Compute the recording tableau in the collapsing procedure (case 1) and collapse the ball arrangement.
    
    def recording_and_collapse(self):
        collapsed_balls = [(self.size[0])*[0]]
        for b in self.balls_coordinates:
            new_row = (self.size[0])*[0]
            new_row[b[1]-1] = 1
            
            collapsed_balls.append(new_row)
            #print(collapsed_balls)
            
            if self.case in [0,1,2]:
                for j in reversed(range(1,len(collapsed_balls))):
                    row1 = collapsed_balls[j-1].copy()
                    row2 = collapsed_balls[j].copy()
                    [r2,r1,wit] = two_row_collapse(row2,row1)
                    if not wit:
                        self.QM[j].append(b[0])
                        break
                    elif j == 1:
                        self.QM[j-1].append(b[0])
                    [collapsed_balls[j],collapsed_balls[j-1]] = [r2,r1]


                while (self.size[0])*[0] in collapsed_balls:
                    collapsed_balls.remove((self.size[0])*[0])
                    
            elif self.case in [3,4]:
                for j in reversed(range(1,len(collapsed_balls))):
                    row1 = collapsed_balls[j-1].copy()
                    row2 = collapsed_balls[j].copy()
                    [r2,r1,wit] = two_row_collapse(row2,row1)
                    if not wit:
                        self.QM[j].append(b[0])
                        break
                    elif j == 1:
                        self.QM[j-1].append(b[0])
                    [collapsed_balls[j],collapsed_balls[j-1]] = [r2,r1]


                while (self.size[0])*[0] in collapsed_balls:
                    collapsed_balls.remove((self.size[0])*[0])
            
            #print(collapsed_balls)
        
        while [] in self.QM:
            self.QM.remove([])
            
        self.balls = collapsed_balls.copy()
    
    # Draw a queue between pi and pf in a drawing win
    # queues must be ordered from top to bottom
    # case : see pairing cases

    def draw_queue(self,win,pi,pf,case,offset,cells,lab):
        r = self.r
        xi = pi[1]-0.5
        yi = pi[0]-0.5
        xf = pf[1]-0.5
        yf = pf[0]-0.5

        eps = 0.01
        
        if case == 0 or case == 1 or case == 2:
            if xi <= xf:
                if cells == "balls":
                    win+= circle((xi+offset,yi), self.r, fill = False, color = Color('black'))
                    win+= circle((xf+offset,yf), self.r, fill = False, color = Color('black'))
                    win+= text(lab,(xi+offset,yi-eps),color = Color('black'),fontsize = 14,rotation = 0) 
                    win+= text(lab,(xf+offset,yf-eps),color = Color('black'),fontsize = 14,rotation = 0) 
                
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('black'))
                    win += line([(xi+offset,yi-0.5+offset),(0,yi-0.5+offset)],color=Color('black'))
                    win += line([(self.size[0],yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('black'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('black'))

                elif cells == "numbers":
                        offset2 = offset / 3.0
                        win += line([(xi+offset2,yi-self.r),(xi+offset2,yi-0.5+offset2)],color=Color('black'))
                        win += line([(xi+offset2,yi-0.5+offset2),(0,yi-0.5+offset2)],color=Color('black'))
                        win += line([(self.size[0],yi-0.5+offset2),(xf+offset2,yi-0.5+offset2)],color=Color('black'))
                        win += line([(xf+offset2,yi-0.5+offset2),(xf+offset2,yf+self.r)],color=Color('black'))

            else:
                if cells == "balls":
                    win+= circle((xi+offset,yi), self.r, fill = False, color = Color('black'))
                    win+= circle((xf+offset,yf), self.r, fill = False, color = Color('black'))
                    win+= text(lab,(xi+offset,yi-eps),color = Color('black'),fontsize = 14,rotation = 0) 
                    win+= text(lab,(xf+offset,yf-eps),color = Color('black'),fontsize = 14,rotation = 0)
                
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('black'))
                    win += line([(xi+offset,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('black'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('black'))

                elif cells == "numbers":
                        offset3 = offset / 3.0
                        win += line([(xi+offset3,yi-self.r),(xi+offset3,yi-0.5+offset3)],color=Color('black'))
                        win += line([(xi+offset3,yi-0.5+offset3),(xf+offset3,yi-0.5+offset3)],color=Color('black'))
                        win += line([(xf+offset3,yi-0.5+offset3),(xf+offset3,yf+self.r)],color=Color('black'))
            
            return(win)
        
        if case == 3 or case == 4:
            if xi <= xf:
                if cells == "balls":
                    win+= circle((xi+offset,yi), self.r, fill = False, color = Color('black'))
                    win+= circle((xf+offset,yf), self.r, fill = False, color = Color('black'))
                    
                win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('black'))
                win += line([(xi+offset,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('black'))
                win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('black'))
            else:
                if cells == "balls":
                    win+= circle((xi+offset,yi), self.r, fill = False, color = Color('black'))
                    win+= circle((xf+offset,yf), self.r, fill = False, color = Color('black'))
                    
                win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('black'))
                win += line([(xi+offset,yi-0.5+offset),(self.size[0],yi-0.5+offset)],color=Color('black'))
                win += line([(0,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('black'))
                win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('black'))
                
            return(win)
            
                    
    # Gives the drawing of a MLQ that can be paired or not (in which case just shows the ball arrangement)             
    def draw(self,radius,cells):
        win = Graphics()
        win.set_aspect_ratio(1)
        self.r = radius
        w = self.size[1]
        h = self.size[0]

        eps = 0.01
        
        #Draw gray grid        
        for i in range(0,h+1,1):
            win += line([(i,0),(i,w)],color=Color('lightgray'))
        for i in range(0,w+1,1):    
            win += line([(0,i),(h,i)],color=Color('lightgray'))
            
        #Some important parameters
        qs = [i for i in self.queues if len(i) > 1]
        sqs = [i for i in self.queues if len(i) == 1]
        N = len(qs)
        #maximum offset to avoid queues to be intersecting in the drawing
        par = (-1)/(8)
        
        #Draw ball arrangement
        for b in self.balls_coordinates:
            yb = b[0]
            xb = b[1]
            num = self.balls_coordinates.count(b)
            
            if cells == "numbers":
                lab = str(num)
                win+= text(lab,(xb-0.5,yb-0.5),color = Color('blue'),fontsize = 20,rotation = 0)
                
                #To add colors to certain rows
                
                #if (yb < 2):
                #    win+= text(lab,(xb-0.5,yb-0.5),color = Color('blue'),fontsize = 18,rotation = 0, fontweight = 'bold')
                #else:
                #    win+= text(lab,(xb-0.5,yb-0.5),color = Color('red'),fontsize = 18,rotation = 0)
            
            if cells == "balls" and len(self.queues) == 0:
                for k in range(num):
                    par_loc = -1/(num+1)
                    off = (par_loc)-(par_loc/(1.0*(N-1)))*(k)
                    win+= circle((xb+off,yb-0.5), self.r, fill = False, color = Color('black'))      
            
        #Draw queues     
        for k in range(len(self.queues)):
            queue = self.queues[k] 
            l = len(queue)
            if cells == "balls":
                if l == 1:
                    num = sqs.count(queue)
                    for k in range(num):
                        off = (par)-(4*par/(1.0*(N-1)))*(k)
                        xb = queue[0][1]
                        yb = queue[0][0]
                        win+= circle((xb+off-0.5,yb-0.5), self.r, fill = False, color = Color('black'))
                        win+= text("1",(xb-0.5+off,yb-0.5-eps),color = Color('black'),fontsize = 14,rotation = 0)
            if l > 1:                    
                for j in range(1,l):                    
                    
                    #offset function
                    off = (par)-(3.5*par/(1.0*(N-1)))*(k)
                    
                    #draw the line between elements of the queue
                    win = self.draw_queue(win,queue[j-1],queue[j],self.case,off,cells,len(queue))

        win.axes(False)
        win.show()
        
    def cw(self):
        word= []
        for i in reversed(range(self.size[0])):
            for j in range(self.size[1]):
                for k in range(self.balls[j][i]):
                    word.append(j+1)
        return(word)
            
    def partition(self):
        lam_prime = [sum(b) for b in self.balls]
        lam_p = Partition(lam_prime)
        return(lam_p.conjugate())
        
    ##  ALGEBRAIC METHODS  ##
    
    #returns the x-weight of the multiline queue. Does not need to be paired
    def xweight(self):
        wei = xs[0]**0
        for b in self.balls_coordinates:
            wei *= xs[b[1]-1]
        return(wei)
    
    #returns the x-weight of the multiline queue. Requires pairing
    # case: see pairing cases
    def qweight(self):
        wei = q**0
        #q statistic calculation depends on the pairing algorithm
        if self.case == 0 or self.case == 1 or self.case == 2:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf >= ci:
                        wei *= q**(l+1-queue[i-1][0])
                        
        elif self.case == 3 or self.case == 4:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf <= ci:
                        wei *= q**(l+1-queue[i-1][0])
        return(wei)
    
    
    def comaj(self):
        wei = q**0
        #q statistic calculation depends on the pairing algorithm
        if self.case == 0 or self.case == 1 or self.case == 2:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf < ci:
                        wei *= q**(l+1-queue[i-1][0])
                        
        elif self.case == 3 or self.case == 4:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf > ci:
                        wei *= q**(l+1-queue[i-1][0])
        return(wei)
        

In [3]:
# Matrix rotations

# rotate the ball arrangement 90 degrees CLOCKWISE
def rotate_clockwise(balls):
    balls2 = [[0 for i in range(len(balls))] for j in range((len(balls[0])))]
    for i in range(len(balls)):
        for j in range(len(balls[0])):
            num = balls[i][j]
            if num != 0:
                balls2[j][i] = num
        
    return(list(reversed(balls2)))

# rotate the ball arrangement 90 degrees COUNTER-CLOCKWISE
def rotate_counterclockwise(balls):
    w = len(balls[0])
    h = len(balls)
    balls2 = [[0 for i in range(h)] for j in range(w)]
    for i in range(h):
        for j in range(w):
            num = balls[i][j]
            if num != 0:
                balls2[j][h-i-1] = num
        
    return(balls2)
    

# balls is a ball arrangement
# return the corresponding pair of multiline queues for the given case of pairing
def dRSK(balls, case): 
    r = 0.15
    balls1 = balls.copy()
    balls2 = rotate_clockwise(balls.copy())
    
    M1 = MultilineDiagram(balls1)
    print("Original integer matrix")
    M1.draw(r,"numbers")
    
    print("Collapse Down")
    M1.pair(2)
    #print(M1.balls)
    M1.draw(r,"numbers")
    print("Recording Tableau")
    #print(M1.QM)
    print_list_of_lists(reversed(M1.QM))
    
    print("\n")
    
    M2 = MultilineDiagram(balls2)
    print("Collapse Left")
    M2.pair(4)
    #print(M2.balls)
    M2.draw(r,"numbers")
    print("Recording Tableau")
    #print(M2.QM)
    print_list_of_lists(reversed(M2.QM))
    print("\n")
    
    # Double collapsing #1
    balls3 = rotate_counterclockwise(M2.balls.copy())
    M3 = MultilineDiagram(balls3)
    print("Collapse Left-Down")
    M3.pair(2)
    M3.draw(r,"numbers")
    print("Recording Tableau")
    print_list_of_lists(reversed(M3.QM))
    print("\n")
    
    # Double collapsing #2
    balls4 = rotate_clockwise(M1.balls.copy())
    M4 = MultilineDiagram(balls4)
    print("Collapse Down-Left")
    M4.pair(4)
    M4.draw(r,"numbers")
    print("Recording Tableau")
    print_list_of_lists(reversed(M4.QM))
    print("\n")
    
    print("Check that the inverse works:")
    balls5 = dRSK_inverse(M1,list(M3.QM),2)
    M5 = MultilineDiagram(balls5)
    M5.draw(r,"numbers")
    
    
# Auxiliary methods for inverse map

# Lift the most exterior (depending on case) ball that is unpaired from row2 to row1

def two_row_lift(row1,row2,case):
    if case == 2:
        par_word = []
        # Create the "inverse" parenthesis word to do the matching algorithm
        for i in reversed(range(len(row1))):
            if row2[i]!=0:
                for k in range(row2[i]):
                    par_word.append(2)
            else:
                par_word.append(0)
            if row1[i]!=0:
                for k in range(row1[i]):
                    par_word.append(1)
            else:
                par_word.append(0)
                
        # Find indices of each row balls
        row1_indices = []
        row2_indices = []

        for j in range(len(par_word)):
            if par_word[j] == 2:
                row2_indices.append(j)
            elif par_word[j] == 1:
                row1_indices.append(j)
                
        # Apply parenthesis matching
        red_par_word = match_parenthesis(par_word)
        
        # Find index on row2 to lift
        lift_indices = []

        j = max([k for k in range(len(red_par_word)) if red_par_word[k] == 2])
        ind = row2_indices.index(j)
        
        pos = max([l for l in range(len(row2)) if sum(row2[l:])>ind])

        row2[pos] -= 1
        row1[pos] += 1  
    
    if case == 4:
        row1.reverse()
        row2.reverse()
        two_row_lift(row1,row2,2)
        row1.reverse()
        row2.reverse()
    
    

# Compute the binary matrix associated to a multiline queue M and a (recording) tableau Qtab
# M and Qtab need have matching sizes
def dRSK_inverse(M,Qtab,case):
        balls = M.balls.copy()
        w = len(balls[0])
        h = len(balls)
        rec = Qtab.copy()
        while rec != []:
            M = max([max(rec[i]) for i in range(len(rec))])
            row = min([j for j in range(len(rec)) if M in rec[j]])
            aux = list(reversed(rec[row]))
            aux.remove(M)
            rec[row] = list(reversed(aux))
            for k in range(M-len(balls)):
                balls.append([0 for i in range(w)])
            for i in range(row,M-1):
                row1 = balls[i+1]
                row2 = balls[i]
                
                #print(row1)
                #print(row2)
                #print("\n")
                
                two_row_lift(row1,row2,case)
                
                #print(row1)
                #print(row2)
                #print("--------------------")
                
            while [] in rec:
                rec.remove([])
        
        return(balls)    

## Algebraic Combinatorics

In [8]:
##   Schur functions   ##

def schur_mld(n,shape):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(shape)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        num_list = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
        aux_list.append([list(sub) for sub in Subsets(num_list,submultiset=True) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)

    # Create all MLDs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        M = MultilineDiagram(balls.copy())
        M.pair(0)
        #print(M.queues)
        if M.qweight() == 1:
            #r=0.2
            #M.draw(r,"numbers")
            poly += M.xweight()

    # Convert polynomial to symmetric function
    f = sym.from_polynomial(poly)
    return(s(f))

## Modified Hall-Littlewood Polynomials ##

def cocharge_gen(n,shape):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(shape)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        num_list = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
        aux_list.append([list(sub) for sub in Subsets(num_list,submultiset=True) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)
    poly = 0

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        
        balls = ball_arr

        # Create the corresponding MLD with n columns of given shape & compute the polynomial

        M = MultilineDiagram(balls.copy())
        M.pair(0)
        
        xM = M.xweight()
        poly += M.comaj()*xM

    return(poly)


##  ???  ##

def charge_gen(n,shape):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(shape)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        num_list = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
        aux_list.append([list(sub) for sub in Subsets(num_list,submultiset=True) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)
    poly = 0

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        
        balls = ball_arr

        # Create the corresponding MLD with n columns of given shape & compute the polynomial

        M = MultilineDiagram(balls.copy())
        M.pair(0)
        
        xM = M.xweight()
        poly += M.qweight()*xM

    return(poly)

# Other methods

def exponent(l):
    ans = xs[0]**0
    for i in range(len(l)):
        m = int(l[i])
        ans = ans*xs[i]**m
    return(ans)


##   Kostka-Foulkes polynomials   ##

def KostKaFoulkesPol_mld(n,lam,mu):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(lam)
    mlq_shape = list(part)

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        num_list = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
        aux_list.append([list(sub) for sub in Subsets(num_list,submultiset=True) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)
    poly = 0

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        
        balls = ball_arr

        # Create the corresponding MLD with n columns of given shape & compute the polynomial

        M = MultilineDiagram(balls.copy())
        M.pair(0)
        
        xM = M.xweight()
        
        if xM == exponent(mu) and M.qweight() == 1:
            #r=0.2
            #M.draw(r)
            b = M.balls.copy()
            bN = rotate_counterclockwise(b)
            N = MultilineDiagram(bN)
            #N.draw(r)
            N.pair(3)
            poly += N.qweight()

    return(poly)



def modifiedKostkaFoulkesPol_mld(n,lam,mu):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(mu)
    mld_shape = list(part)

    aux_list = []
    ball_arrangements = []

    for i in mld_shape:
        num_list = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
        aux_list.append([list(sub) for sub in Subsets(num_list,submultiset=True) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)
    poly = 0

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        
        balls = ball_arr

        # Create all MLQs with n columns of given shape & compute the polynomial

        M = MultilineDiagram(balls.copy())
        M.pair(0)
        
        xM = M.xweight()
        
        if xM == exponent(list(lam)):
            b = M.balls.copy()
            N = MultilineDiagram(b)
            N.pair(2)
            
            if(N.partition() == lam.conjugate()):
                M.draw(0.2,"numbers")
                #N.draw(0.2,"numbers")
                poly += M.comaj()

    # Convert polynomial to symmetric function
    return(poly)
